In [1]:
import pandas as pd

In [2]:
#Read matches.csv data
matches = pd.read_csv("matches.csv", index_col=0)

In [3]:
#See first lines of the data
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal


In [4]:
#See shape of the data
matches.shape

(1386, 27)

In [5]:
#Investigate missing data by seeing how many games each team has
matches["team"].value_counts()

team
Nottingham Forest           70
Aston Villa                 70
Brentford                   70
West Ham United             70
Fulham                      70
Arsenal                     69
Wolverhampton Wanderers     69
Everton                     69
Crystal Palace              69
Liverpool                   69
Bournemouth                 69
Brighton and Hove Albion    69
Newcastle United            69
Manchester United           69
Tottenham Hotspur           69
Manchester City             69
Chelsea                     68
Leicester City              38
Leeds United                38
Southampton                 38
Luton Town                  32
Burnley                     32
Sheffield United            31
Name: count, dtype: int64

In [6]:
#Investigate missing data by seeing matchweeks count
matches["round"].value_counts()

round
Matchweek 1     40
Matchweek 17    40
Matchweek 32    40
Matchweek 31    40
Matchweek 30    40
Matchweek 28    40
Matchweek 27    40
Matchweek 25    40
Matchweek 24    40
Matchweek 23    40
Matchweek 22    40
Matchweek 21    40
Matchweek 2     40
Matchweek 19    40
Matchweek 18    40
Matchweek 20    40
Matchweek 16    40
Matchweek 8     40
Matchweek 3     40
Matchweek 4     40
Matchweek 5     40
Matchweek 15    40
Matchweek 7     40
Matchweek 6     40
Matchweek 9     40
Matchweek 10    40
Matchweek 11    40
Matchweek 12    40
Matchweek 13    40
Matchweek 14    40
Matchweek 26    38
Matchweek 29    28
Matchweek 33    20
Matchweek 34    20
Matchweek 35    20
Matchweek 36    20
Matchweek 37    20
Matchweek 38    20
Name: count, dtype: int64

In [7]:
#Conclusion: Missing data is from delegation/promotion teams and current season missing matches

In [8]:
#Clean-up data
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

In [9]:
#Machine Learning can't work with data tha is object

In [10]:
#Convert date from object to a datetime
matches["date"] = pd.to_datetime(matches["date"])

In [11]:
#Add new column to represent venue as a number
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [12]:
#Add new column to represent opponent as a number
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [13]:
#Add new column to remove the ':' and the minutes from the time
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [14]:
#Add new column to represent each day of the week as a number
matches["day_code"] = matches["date"].dt.dayofweek

In [15]:
#Add target column, identifying if a team Won
matches["target"] = (matches["result"] == "W").astype("int")

In [16]:
#Add new team_name column
matches["team_name"] = matches["team"]

In [17]:
#Start the training model
from sklearn.ensemble import RandomForestClassifier

In [18]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=1)

In [19]:
train = matches[matches["date"] < '2023-01-01']
test = matches[matches["date"] > '2023-01-01']
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [20]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, random_state=1)

In [21]:
preds = rf.predict(test[predictors])

In [22]:
#Do some backtesting of the model
from sklearn.metrics import accuracy_score

In [23]:
accuracy = accuracy_score(test["target"], preds)

In [24]:
accuracy

0.6195445920303605

In [25]:
#See when accuracy was higher vs lower
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,517,130
1,271,136


In [26]:
#Revise accuracy metric
from sklearn.metrics import precision_score

In [27]:
precision_score(test["target"], preds)

0.5112781954887218

In [28]:
#Improve the model precision with rolling averages
grouped_matches = matches.groupby("team")

In [29]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [30]:
cols = ["gf", "ga", "poss", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [31]:
new_cols

['gf_rolling',
 'ga_rolling',
 'poss_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [32]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols), include_groups=False)

In [33]:
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 3  2022-08-27  17:30  Premier League   Matchweek 4   
                        4  2022-08-31  19:30  Premier League   Matchweek 5   
                        5  2022-09-04  16:30  Premier League   Matchweek 6   
                        7  2022-09-18  12:00  Premier League   Matchweek 8   
                        8  2022-10-01  12:30  Premier League   Matchweek 9   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 32 2024-03-02  15:00  Premier League  Matchweek 27   
                        33 2024-03-09  15:00  Premier League  Matchweek 28   
                        35 2024-03-30  17:30  Premier League  Matchweek 30   
                        36 2024-04-02  19:45  Premier League  Matchweek 31   
                        37 2024-04-06  15:00  Premier League  Matchweek 32   

                            day venue result   gf   ga        opponent  ...  \
team                                                                    ...   
Arsenal                 3   Sat  Home      W  2.0  1.0          Fulham  ...   
                        4   Wed  Home      W  2.0  1.0     Aston Villa  ...   
                        5   Sun  Away      L  1.0  3.0  Manchester Utd  ...   
                        7   Sun  Away      W  3.0  0.0       Brentford  ...   
                        8   Sat  Home      W  3.0  1.0       Tottenham  ...   
...                         ...   ...    ...  ...  ...             ...  ...   
Wolverhampton Wanderers 32  Sat  Away      L  0.0  3.0   Newcastle Utd  ...   
                        33  Sat  Home      W  2.0  1.0          Fulham  ...   
                        35  Sat  Away      L  0.0  2.0     Aston Villa  ...   
                        36  Tue  Away      D  1.0  1.0         Burnley  ...   
                        37  Sat  Home      L  1.0  2.0        West Ham  ...   

                                          team_name  gf_rolling  ga_rolling  \
team                                                                          
Arsenal                 3                   Arsenal    3.000000    0.666667   
                        4                   Arsenal    3.000000    1.000000   
                        5                   Arsenal    2.333333    0.666667   
                        7                   Arsenal    1.666667    1.666667   
                        8                   Arsenal    2.000000    1.333333   
...                                             ...         ...         ...   
Wolverhampton Wanderers 32  Wolverhampton Wanderers    1.000000    1.000000   
                        33  Wolverhampton Wanderers    1.000000    1.333333   
                        35  Wolverhampton Wanderers    1.000000    1.333333   
                        36  Wolverhampton Wanderers    0.666667    2.000000   
                        37  Wolverhampton Wanderers    1.000000    1.333333   

                            poss_rolling sh_rolling sot_rolling dist_rolling  \
team                                                                           
Arsenal                 3      50.333333  14.333333    5.000000    14.133333   
                        4      59.333333  18.333333    7.000000    14.433333   
                        5      62.333333  19.333333    7.333333    15.533333   
                        7      63.333333  20.000000    6.333333    16.800000   
                        8      60.666667  17.000000    6.000000    17.700000   
...                                  ...        ...         ...          ...   
Wolverhampton Wanderers 32     54.333333  14.000000    4.666667    15.900000   
                        33     49.333333  12.333333    4.000000    15.466667   
                        35     55.666667  11.000000    2.666667    15.600000   
                        36     52.000000  11.000000    3.000000    15.600000   
  

In [34]:
matches_rolling = matches_rolling.droplevel('team')
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,team_name,gf_rolling,ga_rolling,poss_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Arsenal,3.000000,0.666667,50.333333,14.333333,5.000000,14.133333,0.333333,0.0,0.0
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Arsenal,3.000000,1.000000,59.333333,18.333333,7.000000,14.433333,0.333333,0.0,0.0
5,2022-09-04,16:30,Premier League,Matchweek 6,Sun,Away,L,1.0,3.0,Manchester Utd,...,Arsenal,2.333333,0.666667,62.333333,19.333333,7.333333,15.533333,0.666667,0.0,0.0
7,2022-09-18,12:00,Premier League,Matchweek 8,Sun,Away,W,3.0,0.0,Brentford,...,Arsenal,1.666667,1.666667,63.333333,20.000000,6.333333,16.800000,1.000000,0.0,0.0
8,2022-10-01,12:30,Premier League,Matchweek 9,Sat,Home,W,3.0,1.0,Tottenham,...,Arsenal,2.000000,1.333333,60.666667,17.000000,6.000000,17.700000,0.666667,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,L,0.0,3.0,Newcastle Utd,...,Wolverhampton Wanderers,1.000000,1.000000,54.333333,14.000000,4.666667,15.900000,0.333333,0.0,0.0
33,2024-03-09,15:00,Premier League,Matchweek 28,Sat,Home,W,2.0,1.0,Fulham,...,Wolverhampton Wanderers,1.000000,1.333333,49.333333,12.333333,4.000000,15.466667,0.000000,0.0,0.0
35,2024-03-30,17:30,Premier League,Matchweek 30,Sat,Away,L,0.0,2.0,Aston Villa,...,Wolverhampton Wanderers,1.000000,1.333333,55.666667,11.000000,2.666667,15.600000,0.000000,0.0,0.0
36,2024-04-02,19:45,Premier League,Matchweek 31,Tue,Away,D,1.0,1.0,Burnley,...,Wolverhampton Wanderers,0.666667,2.000000,52.000000,11.000000,3.000000,15.600000,0.333333,0.0,0.0


In [35]:
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,team_name,gf_rolling,ga_rolling,poss_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Arsenal,3.000000,0.666667,50.333333,14.333333,5.000000,14.133333,0.333333,0.0,0.0
1,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Arsenal,3.000000,1.000000,59.333333,18.333333,7.000000,14.433333,0.333333,0.0,0.0
2,2022-09-04,16:30,Premier League,Matchweek 6,Sun,Away,L,1.0,3.0,Manchester Utd,...,Arsenal,2.333333,0.666667,62.333333,19.333333,7.333333,15.533333,0.666667,0.0,0.0
3,2022-09-18,12:00,Premier League,Matchweek 8,Sun,Away,W,3.0,0.0,Brentford,...,Arsenal,1.666667,1.666667,63.333333,20.000000,6.333333,16.800000,1.000000,0.0,0.0
4,2022-10-01,12:30,Premier League,Matchweek 9,Sat,Home,W,3.0,1.0,Tottenham,...,Arsenal,2.000000,1.333333,60.666667,17.000000,6.000000,17.700000,0.666667,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,L,0.0,3.0,Newcastle Utd,...,Wolverhampton Wanderers,1.000000,1.000000,54.333333,14.000000,4.666667,15.900000,0.333333,0.0,0.0
1313,2024-03-09,15:00,Premier League,Matchweek 28,Sat,Home,W,2.0,1.0,Fulham,...,Wolverhampton Wanderers,1.000000,1.333333,49.333333,12.333333,4.000000,15.466667,0.000000,0.0,0.0
1314,2024-03-30,17:30,Premier League,Matchweek 30,Sat,Away,L,0.0,2.0,Aston Villa,...,Wolverhampton Wanderers,1.000000,1.333333,55.666667,11.000000,2.666667,15.600000,0.000000,0.0,0.0
1315,2024-04-02,19:45,Premier League,Matchweek 31,Tue,Away,D,1.0,1.0,Burnley,...,Wolverhampton Wanderers,0.666667,2.000000,52.000000,11.000000,3.000000,15.600000,0.333333,0.0,0.0


In [36]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2023-07-01']
    test = data[data["date"] > '2023-07-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [37]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [38]:
precision

0.5276073619631901

In [39]:
combined

,actual,predicted
35,1,1
36,1,1
37,0,1
38,1,1
39,1,1
...,...,...
1312,0,0
1313,1,1
1314,0,0
1315,0,0


In [40]:
#See teams on combined
combined = combined.merge(matches_rolling[["date", "team_name", "opponent", "result"]], left_index=True, right_index=True)

In [41]:
combined

,actual,predicted,date,team_name,opponent,result
35,1,1,2023-08-12,Arsenal,Nott'ham Forest,W
36,1,1,2023-08-21,Arsenal,Crystal Palace,W
37,0,1,2023-08-26,Arsenal,Fulham,D
38,1,1,2023-09-03,Arsenal,Manchester Utd,W
39,1,1,2023-09-17,Arsenal,Everton,W
...,...,...,...,...,...,...
1312,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L
1313,1,1,2024-03-09,Wolverhampton Wanderers,Fulham,W
1314,0,0,2024-03-30,Wolverhampton Wanderers,Aston Villa,L
1315,0,0,2024-04-02,Wolverhampton Wanderers,Burnley,D


In [42]:
#Combine Home and Away predictions
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)

In [43]:
combined["new_team"] = combined["team_name"].map(mapping)

In [44]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
merged

,actual_x,predicted_x,date,team_name_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_name_y,opponent_y,result_y,new_team_y
0,1,1,2023-08-12,Arsenal,Nott'ham Forest,W,Arsenal,0,0,Nottingham Forest,Arsenal,L,Nottingham Forest
1,1,1,2023-08-21,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
2,0,1,2023-08-26,Arsenal,Fulham,D,Arsenal,0,0,Fulham,Arsenal,D,Fulham
3,1,1,2023-09-03,Arsenal,Manchester Utd,W,Arsenal,0,0,Manchester United,Arsenal,L,Manchester Utd
4,1,1,2023-09-17,Arsenal,Everton,W,Arsenal,0,0,Everton,Arsenal,L,Everton
...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd
545,1,1,2024-03-09,Wolverhampton Wanderers,Fulham,W,Wolves,0,1,Fulham,Wolves,L,Fulham
546,0,0,2024-03-30,Wolverhampton Wanderers,Aston Villa,L,Wolves,1,0,Aston Villa,Wolves,W,Aston Villa
547,0,0,2024-04-02,Wolverhampton Wanderers,Burnley,D,Wolves,0,0,Burnley,Wolves,D,Burnley


In [45]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

actual_x
1    77
0    64
Name: count, dtype: int64

In [47]:
77/(77+64)

0.5460992907801419